In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from contractions import CONTRACTION_MAP
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models import Word2Vec
from gensim import models
import numpy as np
from pattern.en import tag
from nltk.corpus import wordnet as wn
from sklearn import metrics

C:\Users\Dewa Ari\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import pandas as pd

In [3]:
!pip install remove_emoji

In [4]:
import modSpellChecker as sc
character = ['z','y','x','w','v','u','t','s','r','q','p','o','n','m',
            'l','k','j','i','h','g','f','e','d','c','b','a','.',',',';',
            ':','-,','...','?','!','(',')','[',']','{','}','<','>','"','/',
            '\'','#','-','@']

def repeatcharNormalize(text):
    for i in range(len(character)):
        charac_long = 5
        while charac_long >= 2:
            char = character[i]*charac_long
            text = text.replace(char, character[i])
            charac_long -= 1
    return text

def spellNormalize(text):
    spellCheck = []
    for i in text:
        if i not in character:
            j = sc.correction(i)
            spellCheck.append(j)
        else:
            spellCheck.append(i)
    return spellCheck

In [5]:
data = pd.read_csv('dataset_ig_comments.csv')

In [6]:
data[:]

,___typename,comments__data__created_at,comments__data__id,comments__data__owner__id,comments__data__owner__profile_pic_url,comments__data__owner__username,comments__data__text,comments_disabled,dimensions__height,dimensions__width,...,shortcode,tags__-,taken_at_timestamp,thumbnail_resources__config_height,thumbnail_resources__config_width,thumbnail_resources__src,thumbnail_src,urls__-,username,is_spam
0,GraphImage,1541432828,1.796020e+16,2244966035,https://instagram.fsub6-2.fna.fbcdn.net/vp/a28...,prasasty_andini,@rinchanoche gw rasa mukanya ketukerrr :v,False,1350.0,1080.0,...,Bptip2fgIEG,diaryukikato,1.541234e+09,150.0,150.0,https://instagram.fsub6-2.fna.fbcdn.net/vp/834...,https://instagram.fsub6-2.fna.fbcdn.net/vp/936...,https://instagram.fsub6-2.fna.fbcdn.net/vp/362...,yukikt,no
1,NaN,1541433606,1.797060e+16,1189067964,https://instagram.fsub6-2.fna.fbcdn.net/vp/228...,riiiii12_,"@rinchanoche sebenernya bukan tempat rin, tapi...",NaN,NaN,NaN,...,NaN,NaN,NaN,240.0,240.0,https://instagram.fsub6-2.fna.fbcdn.net/vp/636...,NaN,NaN,NaN,no
2,NaN,1541433624,1.791820e+16,1189067964,https://instagram.fsub6-2.fna.fbcdn.net/vp/228...,riiiii12_,"@prasasty_andini iya, gaenak yg ditahan tahan mah",NaN,NaN,NaN,...,NaN,NaN,NaN,320.0,320.0,https://instagram.fsub6-2.fna.fbcdn.net/vp/9de...,NaN,NaN,NaN,no
3,NaN,1541433805,1.796390e+16,1364324512,https://instagram.fsub6-2.fna.fbcdn.net/vp/295...,deliaajulianti,@panisaaa.s ulanggg ee,NaN,NaN,NaN,...,NaN,NaN,NaN,480.0,480.0,https://instagram.fsub6-2.fna.fbcdn.net/vp/cc6...,NaN,NaN,NaN,no
4,NaN,1541434481,1.799200e+16,8888590655,https://instagram.fsub6-2.fna.fbcdn.net/vp/e4d...,queenmode2000,"Menjual baju import branded murah, bisa untuk ...",NaN,NaN,NaN,...,NaN,NaN,NaN,640.0,640.0,https://instagram.fsub6-2.fna.fbcdn.net/vp/936...,NaN,NaN,NaN,yes
5,NaN,1541434558,1.799650e+16,1582937497,https://instagram.fsub6-2.fna.fbcdn.net/vp/75b...,bagus_jez,Pas keliling jangan lupa sama dengerin lagu th...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
6,NaN,1541435815,1.796690e+16,1833409080,https://instagram.fsub6-2.fna.fbcdn.net/vp/0f1...,sriiicahyati_,🔥💖💖,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
7,NaN,1541439030,1.790680e+16,6540645676,https://instagram.fsub6-2.fna.fbcdn.net/vp/b6a...,ynovi2005,@yan_rian34 halu,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
8,NaN,1541460375,1.797820e+16,4169569956,https://instagram.fsub6-2.fna.fbcdn.net/vp/5e7...,yan_rian34,@ynovi2005 hili,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
9,NaN,1541462266,1.799610e+16,6975177460,https://instagram.fsub6-2.fna.fbcdn.net/vp/787...,biutimasker,Manfaat Masker Spirulina:\n~mengencangkan kuli...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes


In [7]:
feature= data.iloc[:,6]
label = data.iloc[:,28]
print(feature[1:51])
print("---------------")
print(label[1:300])

1     @rinchanoche sebenernya bukan tempat rin, tapi...
2     @prasasty_andini iya, gaenak yg ditahan tahan mah
3                                @panisaaa.s ulanggg ee
4     Menjual baju import branded murah, bisa untuk ...
5     Pas keliling jangan lupa sama dengerin lagu th...
6                                                   🔥💖💖
7                                     @yan_rian34  halu
8                                      @ynovi2005  hili
9     Manfaat Masker Spirulina:\n~mengencangkan kuli...
10    fotonya awesomee 🙂 selamat sore yaah\n hadiah ...
11                               Tambah cantik aja kamu
12            candi prambanan tu kab klaten mbak yuu :)
13    hi, mau diet sehat & BAB lancar? cek IG kita ya 😍
14                i love to the moon and never back❤❤❤❤
15                                Lovely ❤️❤️❤️ @yukikt
16    haii sahabat fellass ! Tau ga knp rambut2 arti...
17            Pasminah diamond only 30k, yuk cek ignya😍
18                  Artis artis siapa yg kurang 

In [8]:
stopword_list=nltk.corpus.stopwords.words('indonesian')
wnl=WordNetLemmatizer()

In [9]:
def tokenize_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[token.strip() for token in tokens]
    return tokens

In [10]:
def expand_contractions(text, contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [11]:
# Anotate text tokens with POS Tags

def pos_tag_text(text):
    
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None
        
    tagged_text = tag(text)
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag))
                          for word, pos_tag in 
                          tagged_text]
    return tagged_lower_text

In [12]:
def lemmatize_text(text):
    pos_tagged_text = pos_tag_text(text)
    lemmatized_tokens=[wnl.lemmatize(word, pos_tag) if pos_tag
                      else word for word,
                      pos_tag in pos_tagged_text]
    lemmatize_text=' '.join(lemmatized_tokens)
    return lemmatize_text

In [13]:
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text =' '.join(filtered_tokens)
    return filtered_text

In [14]:
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [15]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [16]:
def normalize_corpus(data, tokenize=False):
    
    normalized_corpus = []
    for text in data:
        text = expand_contractions(text, CONTRACTION_MAP)
        text = lemmatize_text(text)
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        text = remove_emoji(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            normalized_corpus.append(text)
            
    return normalized_corpus

In [17]:
def tfidf_transformer(bow_matrix):
    
    transformer = TfidfTransformer(norm='l1',
                                  smooth_idf=True,
                                  use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix

In [18]:
def tfidf_extractor(data, ngram_range=(1,1)):
    
    vectorizer = TfidfVectorizer(min_df=1,
                                norm='l1',
                                smooth_idf=True,
                                use_idf=True,
                                ngram_range=ngram_range)
    features = vectorizer.fit_transform(data)
    return vectorizer, features

In [19]:
def prepare_datasets(feature, label, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(feature, label,
                                                       test_size=0.17, random_state=42)
    return train_X, test_X, train_Y, test_Y

def remove_empty_docs(feature, label):
    filtered_corpus = []
    filtered_labels = []
    for doc, label in zip(feature, label):
        if doc.strip():
            filtered_corpus.append(doc)
            filtered_labels.append(label)
    return filtered_corpus, filtered_labels

In [20]:
train_corpus, test_corpus, train_labels, test_labels = prepare_datasets(feature, label, test_data_proportion=0.3)

In [21]:
norm_train_corpus = normalize_corpus(train_corpus)
norm_test_corpus = normalize_corpus(test_corpus)
''.strip()

''

In [22]:
norm_train_corpus

['',
 'samping aja cantik bgt  yukikt',
 'inget arti sahabat gak siii ',
 'cantikk   ',
 'fotonya awesomee  selamat sore yaahn hadiah lucu khusus pengajian dll kepoin yaa',
 'komen gw blscuma mimpi buruk yukikt yakan ',
 'ka yuki pngin meet ih',
 'teman2 ibubapak yg lginbutuh dana usaha pengen usahanya tamba maju sy keluarga sy nkoperasi ksp nasari menyediakan peminjaman dana berbasis online diseluruh wilayah indonesia min peminjaman rp5000000 rp250000000 nsyarat yg disediakan nfoto ktp foto kk foto buku rekenin isi formulir peminjaman dananbatas peminjaman 1thn5thn berminat hubungi karyawan ncallwa 0852 3719 7654 chat aja wa amanah real',
 'kaosnya tp ga ukuran xxxl badan yg   ',
 'kak',
 'santokm5 truss maksudnyeeee',
 'lho kak kasi tw ya hp seharihari pakai menghasilkan uang lo ngasilin 725 jt lo kaknnih 1 ribuan orang gabung merasakan nikmatnya lo kak bisnis  nhasil mainin hp membeli barang lo kak  yuk buruan buruan gabung kakak salah satunya orang beruntung   more info 08214793297

In [23]:
norm_test_corpus

['hay kak promo termurah jasa penambahan follower nih murah banget lohh  nsilahkan kak cek ig follower bagus olshoppribadibisnisndmnwa083813787694nbutuh testimoni nih kak',
 'yukikt jogja kak',
 'potonya bagus  🤗',
 'waduhhh ketemu ni waduhh',
 '',
 'laimanga',
 'lucuuuuuuuuu',
 'uuu ghanazz ',
 'cantik bgt  ',
 '    ',
 'kawai  ',
 'kawan2 mahasiswa s1 seperjuangan tolong bantu isi survey linknya profil thanks',
 ' ',
 'pengen ketemu ma yuki',
 '  ',
 'kirakira nambah badan 110 cm seneng gak iya senengnya yuk cek ig ku seneng biiiisa nambah badannya paket produk peninggi punyaku 100 work',
 'alat pembesaaaar payuuuudara nabilaherbalproducts bener bener ampuh payudaraaaaa udah ga longgar pake bra makasih iya say produkkkk nya info langsung aja add whatsapp 6288222223969 bbm',
 'lucu kak',
 'pngn lmar yuki kato tpi sayang orang nya centil gk puny bulu ketek',
 'grosir kartu kuota internet telkomseln15gb 28rbn17gb 33rbn30gb 45rbn37gb 70rbnflash interne 24 jam nall3g4gnminat dmwa',
 'i lo

In [25]:
#bow_vectorizer, bow_train_features = bow_extractor(norm_train_corpus)
#bow_test_features = bow_vectorizer.transform(norm_test_corpus)

tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

tokenized_train = [nltk.word_tokenize(text)
                  for text in norm_train_corpus]

tokenized_test = [nltk.word_tokenize(text)
                 for text in norm_test_corpus]

model = gensim.models.Word2Vec(tokenized_train,
                              size=500,
                              window=100,
                              min_count=30,
                              sample=1e-3)

In [26]:
def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels,
                                              predicted_labels),
                        2))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels,
                                              predicted_labels,
                                               average='weighted'),
                        2))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels,
                                              predicted_labels,
                                               average='weighted'),
                        2))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels,
                                              predicted_labels,
                                               average='weighted'),
                        2))

In [27]:
def train_predict_evaluate_model (classifier, 
                                 train_features, 
                                 train_labels,
                                test_features, 
                                 test_labels):
    classifier.fit(train_features, train_labels)
    predictions = classifier.predict(test_features)
    get_metrics(true_labels = test_labels,
               predicted_labels=predictions)
    return predictions

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

mnb = MultinomialNB()
svm = SGDClassifier(loss='hinge', n_iter=100)


svm_tfidf_predictions = train_predict_evaluate_model(classifier=svm,
                                                  train_features=tfidf_train_features,
                                                  train_labels=train_labels,
                                                  test_features=tfidf_test_features,
                                                  test_labels=test_labels)
print('------------------------------------------')

Accuracy: 0.89
Precision: 0.89
Recall: 0.89
F1 Score: 0.89
------------------------------------------


C:\Users\Dewa Ari\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [29]:
from sklearn.externals import joblib

In [30]:
svm.fit(tfidf_train_features,train_labels)

C:\Users\Dewa Ari\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=100, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [31]:
from sklearn.pipeline import make_pipeline

pipe= make_pipeline(tfidf_vectorizer,svm)

In [32]:
pipe.fit(train_corpus,train_labels)

C:\Users\Dewa Ari\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l1', preprocessor=None, smooth...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [33]:
pipe.score(test_corpus,test_labels)

0.8847926267281107

In [34]:
tempfeature=normalize_corpus(feature)

In [35]:
# datacombine=['Feature:'[tempfeature],'Label:'[label]]
# tempdata =pd.DataFrame(datacombine, columns=['feature','label'])

In [36]:
tempData=pd.DataFrame(np.column_stack([tempfeature,label]),
                     columns=['Feature','Label'])

In [37]:
tempData

,Feature,Label
0,rinchanoche gw mukanya ketukerrr v,no
1,rinchanoche sebenernya rin tergantung orang yg...,no
2,prasastyandini iya gaenak yg ditahan tahan mah,no
3,panisaaas ulanggg ee,no
4,menjual baju import brand murah kuliah kantor ...,yes
5,pas keliling lupa dengerin lagu thailand yg jd...,no
6,,no
7,yanrian34 halu,no
8,ynovi2005 hilus,no
9,manfaat masker spirulina nmengencangkan kulit ...,yes


In [38]:
tempRest=[pipe,tempData]

In [39]:
joblib.dump(tempRest,'InstagramComment.pkl')

['InstagramComment.pkl']